### 复权因子简介
复权因子有多种算法，BaoStock使用的是涨跌幅复权算法。

涨跌幅复权法不是以参加分配的实际收益作为计算前提的，因此无法计算出投资者参加分配的收益。涨跌幅复权法可以计算出原始资金的收益率，假设投资者在除权日前一天卖出全部的股票，然后在除权日用全部资金以昨收盘价买回，即不参加分配。这样的假设基于如下目的，确保初始投入的资金100%得到使用，既不会因为分红而导致投资减少，也不会因为配股而导致投资增加。


#### 根据涨跌幅复权法，复权因子计算方法如下：

后复权因子 = 前一天的收盘价/当天的前收，上市首日后复权因子为1。

前复权因子 = 当天的前收/前一天的收盘价，最近一次除权除息日后的交易日前复权因子为1。

详细文档下载地址：媒体文件:BaoStock复权因子简介.pdf


### 名词解释

除权除息日：除权除息日即股权登记日下一个交易日。在股票的除权除息日，证券交易所都要计算出股票的除权除息价，以作为股民在除权除息日开盘的参考。其意义是股票股利分配给股东的日期。

前复权：以当前股价为基准向前复权计算股价。

后复权：以上市首日股价作为基准向后复权计算股价。

复权因子：复权因子就是权息修复比例。

向前复权因子：以最近一次复权为例，除权除息日复权因子是1，前复权价格和不复权价格一致；股权登记日复权因子小于1，前复权价格是不复权价格乘以当日的前复权因子。

向后复权因子：以最近一次复权为例，除权除息日复权因子和前一交易日不同，后复权价格是不复权价格乘以新的后复权因子；股权登记日复权因子和前一交易日相同，后复权价格是不复权价格乘以旧复权因子。

### 利用复权因子计算复权价格

以浦发银行为例：

#### 浦发银行复权因子

序号	股票代码	复权日期	前复权因子	后复权因子
0	sh.600000	2016-06-23	0.759535	7.128788
1	sh.600000	2017-05-25	1	9.385732

#### 浦发银行不复权数据
序号	date	code	open	close	preclose
0	2017-05-24	sh.600000	15.3800	15.4700	15.4300
1	2017-05-25	sh.600000	11.7500	12.9300	11.7500
2	2017-05-26	sh.600000	12.8100	12.8400	12.9300

#### 浦发银行前复权数据
序号	date	code	open	close	preclose
0	2017-05-24	sh.600000	11.681648	11.750007	11.719625
1	2017-05-25	sh.600000	11.75	12.93	11.75
2	2017-05-26	sh.600000	12.81	12.84	12.93

#### 前复权数据计算方法：

不复权第0行，日期为2017-05-24，开盘价15.38，查找小于等于此日期（早于这个日期）的前复权因子（即2016-06-23的前复权因子）为0.759535，则前复权开盘价为：15.38*0.759535=11.681648；同样的收盘价、昨日收盘价也使用此计算。

不复权第1、2行，日期分别为2017-05-25、2017-05-26，查找小于等于此日期的前复权因子（即2017-05-25的复权因子）为1，则：前复权开盘价 = 不复权开盘价 * 1。 Qfq calculate example.png


#### 浦发银行后复权数据
序号	date	code	open	close	preclose
0	2017-05-24	sh.600000	109.64076	110.28235	109.9972
1	2017-05-25	sh.600000	110.28235	121.35751	110.28235
2	2017-05-26	sh.600000	120.231224	120.512794	121.35751

#### 后复权数据计算方法：

不复权第0行，日期为2017-05-24，开盘价15.38，查找小于等于此日期（早于这个日期）的后复权因子（即2016-06-23的后复权因子）为7.128788，则前复权开盘价为：15.38*7.128788=109.64076；同样的收盘价、昨日收盘价也使用此计算。

不复权第1、2行，日期分别为2017-05-25、2017-05-26，查找小于等于此日期的前复权因子（即2017-05-25的后复权因子）为9.385732，则：前复权开盘价 = 不复权开盘价 * 9.385732。 Hfq calculate example.png

### 关于复权的常见问题

#### 前后复权对收益的影响

如果股票有过配股增发，存在追加投资金额的情况，会使得使用前复权和后复权计算出来的收益率和实际收益率可能不同。

#### 前复权和后复权那哪种方式好

各有优劣，如果是分析短周期数据，前后复权差别并不大；如果分析最近一段时间的数据，用前复权比较合适；如果是分析很长一段时间的数据，尤其是分析上市公司上市以来的所有数据，使用后复权比较合适。

In [76]:
import pandas as pd
import datetime
import os
import baostock as bs
import sys

STOCK_URL = '../static/data/'
init_date = '1999-07-26'
_output = sys.stdout

def get_adjust_factor(code):
    today = datetime.datetime.now().strftime('%Y-%m-%d')
    rs_list = []
    rs_factor = bs.query_adjust_factor(code=code, start_date=init_date, end_date=today)
    while (rs_factor.error_code == '0') & rs_factor.next():
        rs_list.append(rs_factor.get_row_data())
    result_factor = pd.DataFrame(rs_list, columns=rs_factor.fields)
#     print(result_factor)
    return result_factor
    
    

# 生成复权因子数据CSV
def generate_adjust_factor(data_frame):
    data_frame.to_csv(STOCK_URL+"adjust_factor_data.csv", encoding="GBK", index=False, mode='w')
    pass

# 更新复权因子数据
def add_to_adjust_factor(data_frame, code):
    if not os.path.exists('../static/data/adjust_factor_data.csv'):
        generate_adjust_factor(data_frame)
    else:
        temp = data_frame.copy(deep=True)
        temp_list = []
        for i in range(temp.count().code):
            if is_adjust_in_csv(temp.iloc[i]):
                temp_list.append(i)
        result = temp.drop(index=temp_list)
        if result.count().code == 0:
            print(f'{code} 复权因子数据已经存在')
        else:
            result.to_csv(STOCK_URL+"adjust_factor_data.csv", mode='a', header=False, index=False)
            print(f'{code} 复权因子数据已经更新')


def is_adjust_in_csv(data_frame):
    adjust = pd.read_csv(STOCK_URL + 'adjust_factor_data.csv')
    filter = adjust.loc[(adjust.code==data_frame.code)&(adjust.dividOperateDate==data_frame.dividOperateDate)]
    count = filter.count().dividOperateDate
    if count == 1:
        return True
    elif count ==0 :
        return False
    else:
        print('复权因子数据重复，请复核')# TODO 复权因子重复的话去重
        return True

def adjust_factor_up_to_date(code):
    result = get_adjust_factor(code)
    if result.count().code ==0:
        print(f'{code} 无复权因子数据')
        return
    else:
        add_to_adjust_factor(data_frame=result, code=code)
    
def all_adjust_factor_up_to_date():
    code = pd.read_csv(STOCK_URL + 'all_stock.csv', usecols=['code'])
    count = code.count().code
    if count == 0:
        print('指数代码为空，请检查代码')
        return
    else:
        _output.write('\r开始更新复权因子数据。。。')
        for row in code.iterrows():
            adjust_factor_up_to_date(code=row[1].code)
        print('复权因子数据数据更新完毕')
    

path = '../static/data/adjust_factor_data.csv'
bs.login()
# s = get_adjust_factor("sh.600001")
# print(os.path.exists(path))
# print(t)
# os.makedirs(path, mode=0o777)
adjust_factor_up_to_date("sh.600001")
# all_adjust_factor_up_to_date()
bs.logout()

login success!
logout success!
